In [28]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [29]:
tf.__version__

'2.3.1'

In [30]:
df=pd.read_csv('Churn_Modelling.csv')
x=df.iloc[:,3:-1].values
y=df.iloc[:,-1].values


In [31]:
print(x)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [32]:
print(y)

[1 0 1 ... 1 1 0]


In [33]:
#label encoding the gender column
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
x[:,2]=le.fit_transform(x[:,2])

In [34]:
print(x)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


In [35]:
#one hot encoding the geography column
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [36]:
ct=ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[1])],remainder='passthrough')
x=np.array(ct.fit_transform(x))
print(x)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [37]:
#splitting the dataset
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [38]:
#feature scaling
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [39]:
ann=tf.keras.models.Sequential()

In [40]:
#adding the input and first hidden layer
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

In [41]:
#adding the 2nd hidden layer
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

In [42]:
#adding the o/p layer
ann.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

In [43]:
#compliling the ann
ann.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [44]:
ann.fit(x_train,y_train,batch_size=32,epochs=100)

Epoch 1/100
250/250 [==============================] - 0s 670us/step - loss: 0.5372 - accuracy: 0.7741
Epoch 2/100
250/250 [==============================] - 0s 750us/step - loss: 0.4516 - accuracy: 0.8081
Epoch 3/100
250/250 [==============================] - ETA: 0s - loss: 0.4267 - accuracy: 0.81 - 0s 714us/step - loss: 0.4300 - accuracy: 0.8148
Epoch 4/100
250/250 [==============================] - 0s 754us/step - loss: 0.4203 - accuracy: 0.8152
Epoch 5/100
250/250 [==============================] - 0s 752us/step - loss: 0.4144 - accuracy: 0.8164
Epoch 6/100
250/250 [==============================] - 0s 696us/step - loss: 0.4096 - accuracy: 0.8180
Epoch 7/100
250/250 [==============================] - 0s 760us/step - loss: 0.4053 - accuracy: 0.8205
Epoch 8/100
250/250 [==============================] - 0s 750us/step - loss: 0.4011 - accuracy: 0.8221
Epoch 9/100
250/250 [==============================] - 0s 726us/step - loss: 0.3967 - accuracy: 0.8230
Epoch 10/100
250/250 [=========

In [45]:
ann.predict(sc.transform([[1,0,0,600,1,40,3,60000,2,1,1,50000]]))

array([[0.02612743]], dtype=float32)

In [47]:
y_pred=ann.predict(x_test)
print(y_pred)

[[0.22262526]
 [0.4144839 ]
 [0.10485128]
 ...
 [0.14769474]
 [0.17419714]
 [0.1762586 ]]


In [48]:
#if the predicted prob >0.5,it'll be assigned with 1 else 0
y_pred=(y_pred>0.5)

In [49]:
np.concatenate((y_pred.reshape(-1,1),y_test.reshape(-1,1)),axis=1)

array([[0, 0],
       [0, 1],
       [0, 0],
       ...,
       [0, 0],
       [0, 0],
       [0, 0]], dtype=int64)

In [50]:
from sklearn.metrics import confusion_matrix,accuracy_score
cm=confusion_matrix(y_test,y_pred)
print(cm)
accuracy_score(y_test,y_pred)

[[1518   77]
 [ 196  209]]


0.8635